In [20]:
import numpy as np
import IPython
import IPython.display as ipd
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as color
import soundfile
import scipy.signal as sig
import sklearn
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from numpy import expand_dims
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import math
import os
from os import listdir
from os.path import isfile, join
import Augmentor
import sklearn.preprocessing as sk

In [8]:
listdir('datafiles')

['nn04d_20180717_000000.wav',
 'nn04f_20180616_000000.wav',
 'nn05c_20190214_000000.wav',
 'nn05d_20180702_000000.wav',
 'nn05e_20180504_000000.wav',
 'nn05e_20181021_000000.wav',
 'nn06b_20181121_000000.wav',
 'nn06d_20180311_000000.wav',
 'nn06e_20180512_000000.wav',
 'nn06e_20181115_000000.wav',
 'nn06e_20190128_000000.wav',
 'nn06f_20180806_000000.wav',
 'nn07b_20180923_000000.wav',
 'nn08a_20181015_000000.wav',
 'nn08c_20180902_000000.wav',
 'nn10a_20180913_000000.wav',
 'nn10b_20180728_000000.wav']

In [22]:
def normalize(x, axis=0):
    return sk.minmax_scale(x, axis=axis)


def spec_plot2_one(s,samp,filename,i):
    
    try:
        os.mkdir(filename.strip('.wav'))
    except FileExistsError:
        pass


    [b,a] = sig.butter(4,[10/samp, 100/samp],'bandpass','ba') 
    data1 = sig.filtfilt(b, a, s)
    data_norm=normalize(data1)
    
    fig, axs = plt.subplots(figsize=(14, 5)) 
    [f, t, Sxx] = sig.spectrogram(data_norm, samp,'hann',nperseg=samp,noverlap=samp*0.9,nfft = samp*3)
    cmap = plt.get_cmap('Spectral') 
    norm = color.Normalize(vmin=np.amin(Sxx), vmax=np.amax(Sxx))
    
    axs.pcolormesh(t, f, Sxx, cmap=cmap, norm=norm)  
    axs.set_ylim(10,70)
    
    plt.tight_layout()
    plt.ioff()
    fig.savefig(filename.strip('.wav')+'/'+str(i[0])+'_'+str(i[1])+'.png', bbox_inches='tight')
    plt.close()   


In [ ]:
X = pd.read_csv('nn_ele_hb_00-24hr_TrainingSet_v2.txt', sep="\t", header=0) 
offset=5 #5 sec offset
x=0
for file_path in listdir('datafiles')[-7:]:  
    samples,sampling_rate=soundfile.read('datafiles/'+file_path)
    num_rows= np.shape(X[X['filename']==file_path])[0]
    ref_table=X[X['filename']==file_path]
    fmax = ref_table['High Freq (Hz)'].values.tolist()
    t_start=ref_table[ref_table.columns[2]][0:num_rows+1]-offset
    t_start[t_start<0]=0
    t_end=ref_table[ref_table.columns[2]][0:num_rows+1]+ref_table[ref_table.columns[4]][0:num_rows+1]+offset
    print(file_path,len(ref_table))
    x+=len(ref_table)
    
    ind=0
    for i,j in zip(t_start.values.tolist(),t_end.values.tolist()):
        spec_plot2_one(samples[math.floor(i)*sampling_rate:math.ceil(j)*sampling_rate],sampling_rate,file_path,[fmax[ind],ind])
        ind+=1
    ind=0    
    
    p = Augmentor.Pipeline(source_directory= file_path.strip('.wav'),output_directory='transformedimages')
    p.random_distortion(probability=1, grid_width=4, grid_height=4, magnitude=8)
    p.rotate180(probability=0.35)
    p.zoom_random(0.3, percentage_area=0.5)
    p.skew_tilt(0.6,0.3)
    p.random_contrast(probability=0.4, min_factor=0.6, max_factor=1)
    p.random_brightness(probability=0.45, min_factor=0.6, max_factor=1)
    p.process()
           
print('total data points', x)